In [9]:
from pathlib import Path
from lib.sampler import generate_graphs

RULES_PATH = Path("lib/sampling_rules.json")
graphs = generate_graphs(500, rules_json_path=RULES_PATH, seed=42)

In [10]:
import hashlib
from classes.event_graph import EventGraph

def stable_hash(label: str):
    h = hashlib.blake2b(label.encode("utf-8"), digest_size=16).digest()
    return int.from_bytes(h, "little")

def WL_neighborhood_labels(graph: EventGraph, iterations: int = 3, inspect: bool = False):
    labels = {node: stable_hash(graph.nodes[node].event_type) for node in graph.nodes}
    history = [labels.copy()]
    
    for _ in range(iterations):
        new_labels = {}
        for node in graph.nodes:
            nbrs = graph.adj_list.get(node, [])  # safe default
            neighbor_labels = [labels[nbr] for nbr in nbrs if nbr in labels]
            
            # deterministic sort even if labels later become non-strings
            neighbor_labels.sort(key=repr)

            signature = (labels[node], tuple(neighbor_labels))

            if inspect:
                # keep inspection output readable and still a string so next iter is safe
                new_label = signature
            else:
                new_label = stable_hash(repr(signature))
            
            new_labels[node] = new_label
            
        labels = new_labels
        history.append(labels.copy())
        
    return history  # list of dicts for it=0..h

def graph_to_fingerprint(graph, D=1024):
    all_labels = WL_neighborhood_labels(graph)
    fingerprint = [0.0] * D
    for labels in all_labels:          # use 0..h
        for feat in labels.values():
            idx = feat % D
            fingerprint[idx] = 1.0       # or +=1 for counts
    return fingerprint

In [15]:
fingerprints = {str(i): graph_to_fingerprint(g) for i, g in enumerate(graphs)}

In [17]:
from lib.pinecone import vdb

print(fingerprints["1"])

# vectors = [{"id": fid, "values": fingerprint} for fid, fingerprint in fingerprints.items()]

# vdb.upsert(
#   vectors=vectors,
#   namespace="wwmp"
# )



[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0,